MAPA DE Campo de viento y líneas de corriente - 200 hPa

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import xarray as xr

In [ ]:

# ============================================
# 1. CARGAR DATOS
# ============================================
print("=== CARGANDO DATOS ERA5 200 hPa ===")
ds = xr.open_dataset(r"C:\Users\User\Desktop\mapitas\vientolineas.nc")

print("\nInformación del dataset:")
print(ds)

# ============================================
# 2. EXTRAER Y VERIFICAR DATOS
# ============================================
u_data = ds['u'].squeeze()
v_data = ds['v'].squeeze()

# Verificar si hay factores de escala
if 'scale_factor' in ds['u'].attrs:
    scale_factor = ds['u'].attrs['scale_factor']
    add_offset = ds['u'].attrs.get('add_offset', 0)
    print(f"\nAplicando factor de escala: {scale_factor}")
    print(f"Aplicando offset: {add_offset}")
    u_data = u_data * scale_factor + add_offset
    v_data = v_data * scale_factor + add_offset

# ============================================
# 3. OBTENER COORDENADAS
# ============================================
lon_name = next((c for c in ['longitude', 'lon', 'LONGITUDE', 'LON'] if c in ds.coords), 
                next((c for c in ['longitude', 'lon', 'LONGITUDE', 'LON'] if c in ds), 'longitude'))
lat_name = next((c for c in ['latitude', 'lat', 'LATITUDE', 'LAT'] if c in ds.coords),
                next((c for c in ['latitude', 'lat', 'LATITUDE', 'LAT'] if c in ds), 'latitude'))

lon = ds[lon_name].values
lat = ds[lat_name].values

# Convertir longitudes si es necesario
if lon.max() > 180:
    print(f"\nConvirtiendo longitudes 0-360° → -180-180°")
    lon = np.where(lon > 180, lon - 360, lon)
    sort_idx = np.argsort(lon)
    lon = lon[sort_idx]
    u_data = u_data[:, sort_idx]
    v_data = v_data[:, sort_idx]

# ============================================
# 4. DEFINIR REGIÓN
# ============================================
lat_min, lat_max = -60, 20    # Sur a Norte
lon_min, lon_max = -120, 10   # Oeste a Este

print(f"\nRegión seleccionada:")
print(f"  Latitud: {lat_min}° a {lat_max}°")
print(f"  Longitud: {lon_min}° a {lon_max}°")

# Recortar región
lat_idx = np.where((lat >= lat_min) & (lat <= lat_max))[0]
lon_idx = np.where((lon >= lon_min) & (lon <= lon_max))[0]

u_region = u_data.values[np.ix_(lat_idx, lon_idx)] if hasattr(u_data, 'values') else u_data[np.ix_(lat_idx, lon_idx)]
v_region = v_data.values[np.ix_(lat_idx, lon_idx)] if hasattr(v_data, 'values') else v_data[np.ix_(lat_idx, lon_idx)]
lat_region = lat[lat_idx]
lon_region = lon[lon_idx]

# Asegurar que Norte está arriba
if lat_region[0] > lat_region[-1]:
    lat_region = lat_region[::-1]
    u_region = u_region[::-1, :]
    v_region = v_region[::-1, :]

# ============================================
# 5. CALCULAR VELOCIDAD Y AJUSTAR ESCALA
# ============================================
wind_speed = np.sqrt(u_region**2 + v_region**2)

print(f"\nEstadísticas del viento:")
print(f"  Mínimo: {wind_speed.min():.2f} m/s")
print(f"  Máximo: {wind_speed.max():.2f} m/s")
print(f"  Promedio: {wind_speed.mean():.2f} m/s")
print(f"  Mediana: {np.median(wind_speed):.2f} m/s")

# Determinar límites óptimos para la barra de colores
vmin = np.percentile(wind_speed, 1)  # Percentil 1%
vmax = np.percentile(wind_speed, 99)  # Percentil 99%

print(f"\nEscala de colores:")
print(f"  vmin: {vmin:.1f} m/s")
print(f"  vmax: {vmax:.1f} m/s")

# ============================================
# 6. CREAR MAPA PRINCIPAL
# ============================================
fig = plt.figure(figsize=(16, 12))
ax = plt.axes(projection=ccrs.PlateCarree())
plt.tight_layout(rect=[0.05, 0.05, 0.95, 0.95])  # 5% de margen
# Características geográficas
ax.add_feature(cfeature.COASTLINE, linewidth=0.9, zorder=4)
ax.add_feature(cfeature.BORDERS, linewidth=0.7, alpha=0.8, zorder=4)
ax.add_feature(cfeature.LAND, facecolor='lightgray', alpha=0.3, zorder=1)
ax.add_feature(cfeature.OCEAN, facecolor='lightblue', alpha=0.2, zorder=1)

# Añadir países de Sudamérica
try:
    import cartopy.io.shapereader as shpreader
    countries = shpreader.natural_earth(resolution='50m',
                                        category='cultural',
                                        name='admin_0_countries')
    reader = shpreader.Reader(countries)
    for country in reader.records():
        # Filtrar países relevantes para la región
        country_name = country.attributes.get('NAME', '')
        if any(name in country_name for name in ['Argentina', 'Brazil', 'Chile', 'Peru', 
                                                'Bolivia', 'Colombia', 'Ecuador', 'Venezuela',
                                                'Uruguay', 'Paraguay', 'Mexico', 'Panama']):
            ax.add_geometries([country.geometry], ccrs.PlateCarree(),
                              facecolor='none', edgecolor='darkgray',
                              linewidth=0.4, alpha=0.6, zorder=2)
except:
    print("  Nota: No se pudieron cargar límites detallados de países")

# Cuadrícula
gl = ax.gridlines(draw_labels=True, linewidth=0.6, alpha=0.6,
                  linestyle='--', color='gray', zorder=3)
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {'size': 10, 'color': 'black'}
gl.ylabel_style = {'size': 10, 'color': 'black'}

# ============================================
# 7. CREAR MALLA
# ============================================
lon_grid, lat_grid = np.meshgrid(lon_region, lat_region)

# ============================================
# 8. FONDO: VELOCIDAD DEL VIENTO (pcolormesh)
# ============================================
print("\nAgregando fondo de velocidad del viento...")

# Fondo con pcolormesh
pcm = ax.pcolormesh(lon_grid, lat_grid, wind_speed,
                    transform=ccrs.PlateCarree(),
                    shading='auto',
                    cmap="viridis",
                    alpha=0.7,
                    vmin=vmin,
                    vmax=vmax,
                    zorder=1)

# Barra de color para el FONDO
cbar_fondo = plt.colorbar(pcm, ax=ax, 
                          orientation='horizontal',
                          pad= 0.0000002,
                          shrink=0.72,
                          extend='both')
cbar_fondo.set_label('Velocidad - Fondo (m/s)', 
                     fontsize=12, 
                     fontweight='bold',
                     labelpad=10)
cbar_fondo.ax.tick_params(labelsize=9)

# ============================================
# 9. LÍNEAS DE CORRIENTE CON COLORES 
# ============================================
print("Generando líneas de corriente coloreadas...")

# LÍNEAS COLOR SEGÚN VELOCIDAD - ¡ASÍ COMO EN TU CÓDIGO ORIGINAL!
strm = ax.streamplot(lon_grid, lat_grid, u_region, v_region,
                     color=wind_speed,          # ← COLOREADAS POR VELOCIDAD
                     cmap='plasma',            # ← RAINBOW COMO ANTES
                     norm=plt.Normalize(vmin=vmin, vmax=vmax),
                     linewidth=1.5,
                     arrowsize=1.5,
                     arrowstyle='->',
                     density=2.0,
                     minlength=0.2,
                     transform=ccrs.PlateCarree(),
                     zorder=2)

# Barra de color para las LÍNEAS
cbar_lineas = plt.colorbar(strm.lines, ax=ax,
                           orientation='horizontal',
                           pad=0.05,
                           shrink=0.72,
                           extend='both')
cbar_lineas.set_label('Velocidad - Líneas (m/s)', 
                      fontsize=12, 
                      fontweight='bold',
                      labelpad=10)
cbar_lineas.ax.tick_params(labelsize=9)

# ============================================
# 10. AGREGAR VECTORES DE VIENTO (FLECHAS)
# ============================================
print("Agregando vectores de viento...")

# DENSIDAD DE FLECHAS
skip = 25  # 1 cada 25 puntos

lon_skip = lon_grid[::skip, ::skip]
lat_skip = lat_grid[::skip, ::skip]
u_skip = u_region[::skip, ::skip]
v_skip = v_region[::skip, ::skip]

# Determinar escala automática
max_speed = wind_speed.max()
scale_value = 800 if max_speed > 50 else 500 if max_speed > 30 else 300 if max_speed > 20 else 200

print(f"  Densidad de flechas: 1 cada {skip} puntos")
print(f"  Escala de flechas: {scale_value}")

# Flechas de viento
quiver = ax.quiver(lon_skip, lat_skip,
                   u_skip, v_skip,
                   transform=ccrs.PlateCarree(),
                   scale=scale_value,
                   width=0.001,
                   color='white',
                   alpha=0.9,
                   zorder=4)

# Referencia de escala
qk = ax.quiverkey(quiver, 0.92, 0.02, 20, 
                  '20 m/s', 
                  labelpos='E',
                  coordinates='axes',
                  fontproperties={'size': 10, 'weight': 'bold'},
                  color='black')

# ============================================
# 11. TÍTULO Y ETIQUETAS
# ============================================
# Título principal
title = 'Campo de viento y líneas de corriente a 200 hPa\n'
title += '23 - 01 - 2026, 00:00 UTC - ERA5'
plt.title(title, fontsize=18, fontweight='bold', pad=20)


# ============================================
# 12. ESTABLECER LÍMITES
# ============================================
ax.set_extent([lon_min, lon_max, lat_min, lat_max], crs=ccrs.PlateCarree())

# ============================================
# 13. GUARDAR Y MOSTRAR
# ============================================
plt.tight_layout(rect=[0, 0.03, 1, 0.92])

output_file = r"C:\Users\User\Desktop\mapitas\viento_200hpa_coloreado.png"
plt.savefig(output_file, dpi=300, bbox_inches='tight', facecolor='white')
print(f"\n✓ Mapa guardado en: {output_file}")

plt.show()



MAPA DE LÍNEAS DE CORRIENTE - 200 hPa

In [ ]:


import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import xarray as xr

# ============================================
# 1. CARGAR DATOS
# ============================================
print("=== CARGANDO DATOS ERA5 200 hPa ===")
ds = xr.open_dataset(r"C:\Users\User\Desktop\mapitas\vientolineas.nc")

print("\nInformación del dataset:")
print(ds)

# ============================================
# 2. EXTRAER Y VERIFICAR DATOS
# ============================================
u_data = ds['u'].squeeze()
v_data = ds['v'].squeeze()

# Verificar si hay factores de escala
if 'scale_factor' in ds['u'].attrs:
    scale_factor = ds['u'].attrs['scale_factor']
    add_offset = ds['u'].attrs.get('add_offset', 0)
    print(f"\nAplicando factor de escala: {scale_factor}")
    print(f"Aplicando offset: {add_offset}")
    u_data = u_data * scale_factor + add_offset
    v_data = v_data * scale_factor + add_offset

# ============================================
# 3. OBTENER COORDENADAS
# ============================================
lon_name = next((c for c in ['longitude', 'lon', 'LONGITUDE', 'LON'] if c in ds.coords), 
                next((c for c in ['longitude', 'lon', 'LONGITUDE', 'LON'] if c in ds), 'longitude'))
lat_name = next((c for c in ['latitude', 'lat', 'LATITUDE', 'LAT'] if c in ds.coords),
                next((c for c in ['latitude', 'lat', 'LATITUDE', 'LAT'] if c in ds), 'latitude'))

lon = ds[lon_name].values
lat = ds[lat_name].values

# Convertir longitudes si es necesario
if lon.max() > 180:
    print(f"\nConvirtiendo longitudes 0-360° → -180-180°")
    lon = np.where(lon > 180, lon - 360, lon)
    sort_idx = np.argsort(lon)
    lon = lon[sort_idx]
    u_data = u_data[:, sort_idx]
    v_data = v_data[:, sort_idx]

# ============================================
# 4. DEFINIR REGIÓN
# ============================================
lat_min, lat_max = -60, 20    # Sur a Norte
lon_min, lon_max = -120, 10   # Oeste a Este

print(f"\nRegión seleccionada:")
print(f"  Latitud: {lat_min}° a {lat_max}°")
print(f"  Longitud: {lon_min}° a {lon_max}°")

# Recortar región
lat_idx = np.where((lat >= lat_min) & (lat <= lat_max))[0]
lon_idx = np.where((lon >= lon_min) & (lon <= lon_max))[0]

u_region = u_data.values[np.ix_(lat_idx, lon_idx)] if hasattr(u_data, 'values') else u_data[np.ix_(lat_idx, lon_idx)]
v_region = v_data.values[np.ix_(lat_idx, lon_idx)] if hasattr(v_data, 'values') else v_data[np.ix_(lat_idx, lon_idx)]
lat_region = lat[lat_idx]
lon_region = lon[lon_idx]

# Asegurar que Norte está arriba
if lat_region[0] > lat_region[-1]:
    lat_region = lat_region[::-1]
    u_region = u_region[::-1, :]
    v_region = v_region[::-1, :]

# ============================================
# 5. CALCULAR VELOCIDAD Y AJUSTAR ESCALA
# ============================================
wind_speed = np.sqrt(u_region**2 + v_region**2)

print(f"\nEstadísticas del viento:")
print(f"  Mínimo: {wind_speed.min():.2f} m/s")
print(f"  Máximo: {wind_speed.max():.2f} m/s")
print(f"  Promedio: {wind_speed.mean():.2f} m/s")
print(f"  Mediana: {np.median(wind_speed):.2f} m/s")

# Determinar límites óptimos para la barra de colores
vmin = np.percentile(wind_speed, 1)  # Percentil 1%
vmax = np.percentile(wind_speed, 99)  # Percentil 99%

print(f"\nEscala de colores:")
print(f"  vmin: {vmin:.1f} m/s")
print(f"  vmax: {vmax:.1f} m/s")

# ============================================
# 6. CREAR MAPA PRINCIPAL
# ============================================
fig = plt.figure(figsize=(16, 12))
ax = plt.axes(projection=ccrs.PlateCarree())
plt.tight_layout(rect=[0.05, 0.05, 0.95, 0.95])  # 5% de margen


# Primero agregar el océano con color celeste claro
ax.add_feature(cfeature.OCEAN, 
               facecolor='#E0F7FF',  # Celeste muy claro
               alpha=0.9,           # Bastante opaco
               zorder=0)

# Luego agregar la tierra con color crema muy claro
ax.add_feature(cfeature.LAND, 
               facecolor='#F5F5DC',  # Beige muy claro (color crema)
               alpha=0.7,           # Semi-transparente
               zorder=1)


# Características geográficas (MÁS VISIBLES SIN FONDO)
ax.add_feature(cfeature.COASTLINE, linewidth=1.2, zorder=4)
ax.add_feature(cfeature.BORDERS, linewidth=0.9, alpha=0.9, zorder=4)
ax.add_feature(cfeature.LAND, facecolor='lightgray', alpha=0.2, zorder=1)
ax.add_feature(cfeature.OCEAN, facecolor='lightblue', alpha=0.1, zorder=1)

# Añadir países de Sudamérica
try:
    import cartopy.io.shapereader as shpreader
    countries = shpreader.natural_earth(resolution='50m',
                                        category='cultural',
                                        name='admin_0_countries')
    reader = shpreader.Reader(countries)
    for country in reader.records():
        # Filtrar países relevantes para la región
        country_name = country.attributes.get('NAME', '')
        if any(name in country_name for name in ['Argentina', 'Brazil', 'Chile', 'Peru', 
                                                'Bolivia', 'Colombia', 'Ecuador', 'Venezuela',
                                                'Uruguay', 'Paraguay', 'Mexico', 'Panama']):
            ax.add_geometries([country.geometry], ccrs.PlateCarree(),
                              facecolor='none', edgecolor='darkgray',
                              linewidth=0.5, alpha=0.7, zorder=2)
except:
    print("  Nota: No se pudieron cargar límites detallados de países")

# Cuadrícula
gl = ax.gridlines(draw_labels=True, linewidth=0.6, alpha=0.6,
                  linestyle='--', color='gray', zorder=3)
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {'size': 10, 'color': 'black'}
gl.ylabel_style = {'size': 10, 'color': 'black'}

# ============================================
# 7. CREAR MALLA
# ============================================
lon_grid, lat_grid = np.meshgrid(lon_region, lat_region)

# ============================================
# 8. LÍNEAS DE CORRIENTE (PRINCIPALES - SIN FONDO)
# ============================================
print("Generando líneas de corriente coloreadas...")

# LÍNEAS COLOR SEGÚN VELOCIDAD - ¡ASÍ COMO EN TU CÓDIGO ORIGINAL!
strm = ax.streamplot(lon_grid, lat_grid, u_region, v_region,
                     color=wind_speed,          # ← COLOREADAS POR VELOCIDAD
                     cmap='plasma',             # ← RAINBOW COMO ANTES
                     norm=plt.Normalize(vmin=vmin, vmax=vmax),
                     linewidth=1.5,             # ← LÍNEAS MÁS GRUESAS PARA MEJOR VISIBILIDAD
                     arrowsize=1.5,
                     arrowstyle='->',
                     density=4,
                     minlength=0.2,
                     transform=ccrs.PlateCarree(),
                     zorder=3)

# Barra de color para las LÍNEAS
cbar_lineas = plt.colorbar(strm.lines, ax=ax,
                           orientation='horizontal',
                           pad=0.05,
                           shrink=0.72,
                           extend='both')
cbar_lineas.set_label('Velocidad del Viento (m/s)', 
                      fontsize=12, 
                      fontweight='bold',
                      labelpad=10)
cbar_lineas.ax.tick_params(labelsize=9)


# ============================================
# 10. TÍTULO Y ETIQUETAS
# ============================================
# Título principal
title = 'Líneas de corriente a 200 hPa\n'
title += '23 - 01 - 2026, 00:00 UTC - ERA5'
plt.title(title, fontsize=18, fontweight='bold', pad=20)

# ============================================
# 11. ESTABLECER LÍMITES
# ============================================
ax.set_extent([lon_min, lon_max, lat_min, lat_max], crs=ccrs.PlateCarree())

# ============================================
# 12. GUARDAR Y MOSTRAR
# ============================================
plt.tight_layout(rect=[0, 0.03, 1, 0.92])

output_file = r"C:\Users\User\Desktop\mapitas\solo_lineas_corriente.png"
plt.savefig(output_file, dpi=300, bbox_inches='tight', facecolor='white')
print(f"\n✓ Mapa guardado en: {output_file}")

plt.show()



MAPA DE CAMPO DE VIENTO 200 hPa

In [ ]:


import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import xarray as xr

# ============================================
# 1. CARGAR DATOS
# ============================================
print("=== CARGANDO DATOS ERA5 200 hPa ===")
ds = xr.open_dataset(r"C:\Users\User\Desktop\mapitas\vientolineas.nc")

print("\nInformación del dataset:")
print(ds)

# ============================================
# 2. EXTRAER Y VERIFICAR DATOS
# ============================================
u_data = ds['u'].squeeze()
v_data = ds['v'].squeeze()

# Verificar si hay factores de escala
if 'scale_factor' in ds['u'].attrs:
    scale_factor = ds['u'].attrs['scale_factor']
    add_offset = ds['u'].attrs.get('add_offset', 0)
    print(f"\nAplicando factor de escala: {scale_factor}")
    print(f"Aplicando offset: {add_offset}")
    u_data = u_data * scale_factor + add_offset
    v_data = v_data * scale_factor + add_offset

# ============================================
# 3. OBTENER COORDENADAS
# ============================================
lon_name = next((c for c in ['longitude', 'lon', 'LONGITUDE', 'LON'] if c in ds.coords), 
                next((c for c in ['longitude', 'lon', 'LONGITUDE', 'LON'] if c in ds), 'longitude'))
lat_name = next((c for c in ['latitude', 'lat', 'LATITUDE', 'LAT'] if c in ds.coords),
                next((c for c in ['latitude', 'lat', 'LATITUDE', 'LAT'] if c in ds), 'latitude'))

lon = ds[lon_name].values
lat = ds[lat_name].values

# Convertir longitudes si es necesario
if lon.max() > 180:
    print(f"\nConvirtiendo longitudes 0-360° → -180-180°")
    lon = np.where(lon > 180, lon - 360, lon)
    sort_idx = np.argsort(lon)
    lon = lon[sort_idx]
    u_data = u_data[:, sort_idx]
    v_data = v_data[:, sort_idx]

# ============================================
# 4. DEFINIR REGIÓN
# ============================================
lat_min, lat_max = -60, 20    # Sur a Norte
lon_min, lon_max = -120, 10   # Oeste a Este

print(f"\nRegión seleccionada:")
print(f"  Latitud: {lat_min}° a {lat_max}°")
print(f"  Longitud: {lon_min}° a {lon_max}°")

# Recortar región
lat_idx = np.where((lat >= lat_min) & (lat <= lat_max))[0]
lon_idx = np.where((lon >= lon_min) & (lon <= lon_max))[0]

u_region = u_data.values[np.ix_(lat_idx, lon_idx)] if hasattr(u_data, 'values') else u_data[np.ix_(lat_idx, lon_idx)]
v_region = v_data.values[np.ix_(lat_idx, lon_idx)] if hasattr(v_data, 'values') else v_data[np.ix_(lat_idx, lon_idx)]
lat_region = lat[lat_idx]
lon_region = lon[lon_idx]

# Asegurar que Norte está arriba
if lat_region[0] > lat_region[-1]:
    lat_region = lat_region[::-1]
    u_region = u_region[::-1, :]
    v_region = v_region[::-1, :]

# ============================================
# 5. CALCULAR VELOCIDAD Y AJUSTAR ESCALA
# ============================================
wind_speed = np.sqrt(u_region**2 + v_region**2)

print(f"\nEstadísticas del viento:")
print(f"  Mínimo: {wind_speed.min():.2f} m/s")
print(f"  Máximo: {wind_speed.max():.2f} m/s")
print(f"  Promedio: {wind_speed.mean():.2f} m/s")
print(f"  Mediana: {np.median(wind_speed):.2f} m/s")

# Determinar límites óptimos para la barra de colores
vmin = np.percentile(wind_speed, 1)  # Percentil 1%
vmax = np.percentile(wind_speed, 99)  # Percentil 99%

print(f"\nEscala de colores:")
print(f"  vmin: {vmin:.1f} m/s")
print(f"  vmax: {vmax:.1f} m/s")

# ============================================
# 6. CREAR MAPA PRINCIPAL
# ============================================
fig = plt.figure(figsize=(16, 12))
ax = plt.axes(projection=ccrs.PlateCarree())
plt.tight_layout(rect=[0.05, 0.05, 0.95, 0.95])  # 5% de margen

# Características geográficas
ax.add_feature(cfeature.COASTLINE, linewidth=0.9, zorder=4)
ax.add_feature(cfeature.BORDERS, linewidth=0.7, alpha=0.8, zorder=4)
ax.add_feature(cfeature.LAND, facecolor='lightgray', alpha=0.3, zorder=1)
ax.add_feature(cfeature.OCEAN, facecolor='lightblue', alpha=0.2, zorder=1)

# Añadir países de Sudamérica
try:
    import cartopy.io.shapereader as shpreader
    countries = shpreader.natural_earth(resolution='50m',
                                        category='cultural',
                                        name='admin_0_countries')
    reader = shpreader.Reader(countries)
    for country in reader.records():
        # Filtrar países relevantes para la región
        country_name = country.attributes.get('NAME', '')
        if any(name in country_name for name in ['Argentina', 'Brazil', 'Chile', 'Peru', 
                                                'Bolivia', 'Colombia', 'Ecuador', 'Venezuela',
                                                'Uruguay', 'Paraguay', 'Mexico', 'Panama']):
            ax.add_geometries([country.geometry], ccrs.PlateCarree(),
                              facecolor='none', edgecolor='darkgray',
                              linewidth=0.4, alpha=0.6, zorder=2)
except:
    print("  Nota: No se pudieron cargar límites detallados de países")

# Cuadrícula
gl = ax.gridlines(draw_labels=True, linewidth=0.6, alpha=0.6,
                  linestyle='--', color='gray', zorder=3)
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {'size': 10, 'color': 'black'}
gl.ylabel_style = {'size': 10, 'color': 'black'}

# ============================================
# 7. CREAR MALLA
# ============================================
lon_grid, lat_grid = np.meshgrid(lon_region, lat_region)

# ============================================
# 8. FONDO: VELOCIDAD DEL VIENTO (pcolormesh) - PRINCIPAL
# ============================================
print("\nAgregando fondo de velocidad del viento...")

# Fondo con pcolormesh (ESTE ES EL ELEMENTO PRINCIPAL AHORA)
pcm = ax.pcolormesh(lon_grid, lat_grid, wind_speed,
                    transform=ccrs.PlateCarree(),
                    shading='auto',
                    cmap="viridis",
                    alpha=0.85,                  # ← ALPHA MAYOR PARA MEJOR VISIBILIDAD
                    vmin=vmin,
                    vmax=vmax,
                    zorder=1)

# Barra de color para el FONDO (ÚNICA BARRA)
cbar_fondo = plt.colorbar(pcm, ax=ax, 
                          orientation='horizontal',
                          pad=0.05,
                          shrink=0.72,
                          extend='both')
cbar_fondo.set_label('Velocidad del Viento (m/s)', 
                     fontsize=12, 
                     fontweight='bold',
                     labelpad=10)
cbar_fondo.ax.tick_params(labelsize=9)

# ============================================
# 9. AGREGAR VECTORES DE VIENTO (FLECHAS)
# ============================================
print("Agregando vectores de viento...")

# DENSIDAD DE FLECHAS
skip = 24  # 1 cada 25 puntos

lon_skip = lon_grid[::skip, ::skip]
lat_skip = lat_grid[::skip, ::skip]
u_skip = u_region[::skip, ::skip]
v_skip = v_region[::skip, ::skip]

# Determinar escala automática
max_speed = wind_speed.max()
scale_value = 800 if max_speed > 50 else 500 if max_speed > 30 else 300 if max_speed > 20 else 200

print(f"  Densidad de flechas: 1 cada {skip} puntos")
print(f"  Escala de flechas: {scale_value}")

# Flechas de viento (COLOR BLANCO PARA CONTRASTAR CON FONDO)
quiver = ax.quiver(lon_skip, lat_skip,
                   u_skip, v_skip,
                   transform=ccrs.PlateCarree(),
                   scale=scale_value,
                   width=0.002,                # ← LIGERAMENTE MÁS GRUESAS
                   color='white',
                   alpha=0.95,                  # ← MÁS OPACAS
                   zorder=4)

# Referencia de escala
qk = ax.quiverkey(quiver, 0.92, 0.02, 20, 
                  '20 m/s', 
                  labelpos='E',
                  coordinates='axes',
                  fontproperties={'size': 10, 'weight': 'bold'},
                  color='black')

# ============================================
# 10. TÍTULO Y ETIQUETAS
# ============================================
# Título principal
title = 'Campo de viento a 200 hPa\n'
title += '23 - 01 - 2026, 00:00 UTC - ERA5'
plt.title(title, fontsize=18, fontweight='bold', pad=20)

# ============================================
# 11. ESTABLECER LÍMITES
# ============================================
ax.set_extent([lon_min, lon_max, lat_min, lat_max], crs=ccrs.PlateCarree())

# ============================================
# 12. GUARDAR Y MOSTRAR
# ============================================
plt.tight_layout(rect=[0, 0.03, 1, 0.92])

output_file = r"C:\Users\User\Desktop\mapitas\viento_con_fondo.png"
plt.savefig(output_file, dpi=300, bbox_inches='tight', facecolor='white')
print(f"\n✓ Mapa guardado en: {output_file}")

plt.show()
